# Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from glob import glob
from IPython.core.magic import register_line_cell_magic

In [ ]:
!unzip /content/drive/MyDrive/Academy/2_DSBA/3_Pre_PJT/Tools/Vision/Data/data_V3.zip;

In [ ]:
train_img = glob('/content/data_V3/images/train/*.jpg')
val_img = glob('/content/data_V3/images/val/*.jpg')
test_img = glob('/content/data_V3/images/test/*.jpg')

train_lab = glob('/content/data_V3/labels/train/*.txt')
val_lab = glob('/content/data_V3/labels/val/*.txt')
test_lab = glob('/content/data_V3/labels/test/*.txt')

print('dataset 구성')
print('train img length:', len(train_img))
print('val img length:', len(val_img))
print('test img length:', len(test_img))

print('train lab length:', len(train_lab))
print('val lab length:', len(val_lab))
print('test lab length:', len(test_lab))

In [ ]:
with open('/content/data_V3/train.txt', 'w')as f:
  f.write('\n'.join(train_img) + '\n')

with open('/content/data_V3/val.txt', 'w')as f:
  f.write('\n'.join(val_img) + '\n')

with open('/content/data_V3/test.txt', 'w')as f:
  f.write('\n'.join(test_img) + '\n')

In [ ]:
@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/data_V2/data.yaml

train: /content/data_V3/train.txt
val: /content//data_V3/val.txt
test: /content/data_V3/test.txt

nc: 1
names: ['person']

In [ ]:
%cat /content/data_V3/data.yaml

#V5

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd /content/yolov5/
!pip install -qr requirements.txt

In [ ]:
%%writetemplate /content/yolov5/models/custom_model.yaml

nc: 1  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_model.yaml

In [ ]:
%cd /content/yolov5/
!python train.py --img 640 --batch 32 --epochs 100 --data /content/drive/MyDrive/Academy/Tools/data_V3/data.yaml --cfg /content/yolov5/models/custom_model.yaml --weights yolov5s.pt --name train_results

In [ ]:
!zip -r /content/drive/MyDrive/Academy/Tools/train_results1.zip /content/yolov5/runs/train/train_results

# V7

In [ ]:
%cd /content
!git clone https://github.com/WongKinYiu/yolov7.git
%cd /content/yolov7/
!pip install -qr requirements.txt

In [ ]:
%%writetemplate /content/yolov7/custom_model.yaml

In [ ]:
%cat /content/yolov7/models/custom_model.yaml

In [ ]:
!python train.py --img 640 --batch 16 --epochs 40 --data /content/drive/MyDrive/Academy/Tools/data_V3/data.yaml --cfg  /content/yolov7/models/custom_model.yaml --weights yolov7.pt --name train_results --cache

#V8

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.193 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.7/166.8 GB disk)


In [ ]:
!yolo train model=yolov8s.pt data=/content/data_V3/data.yaml epochs=50 imgsz=640

In [ ]:
!zip -r /content/drive/MyDrive/Academy/2_DSBA/3_Pre_PJT/Tools/Vision /runs/detect/train